In [55]:
import tensorflow as tf
import tensorflow.sparse as sparse

In [126]:
tf.keras.layers.Pooling2D??

Object `tf.keras.layers.Pooling2D` not found.


In [13]:
x = tf.reshape(tf.range([3*3*2], dtype=tf.float32), shape=(3,3,2))
x[...,0]

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 0.,  2.,  4.],
       [ 6.,  8., 10.],
       [12., 14., 16.]], dtype=float32)>

In [10]:
kernel = tf.constant([[[1,2], [3,4]], [[5,6], [7,8]]], dtype=tf.float32)
kernel[...,0]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 3.],
       [5., 7.]], dtype=float32)>

In [100]:
def get_new_dim(x_dims, kernel_dims, strides):
    return ((x_dims - kernel_dims) // strides) + 1
strides = 1
result_shape = tf.concat([
    x.shape[:-3],
    get_new_dim(tf.constant(x.shape[-3:-1]), tf.constant(kernel.shape[-3:-1]), strides)
], 0)
# tf.tensordot(map1, kernel, axis=)
result_shape

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([2, 2])>

In [26]:
x_tansf_shape = tf.concat([x.shape[:-3], [x.shape[-3]*x.shape[-2]], x.shape[-1:]], 0)
x_transf = tf.reshape(x, shape=x_tansf_shape)
x_transf

<tf.Tensor: shape=(9, 2), dtype=float32, numpy=
array([[ 0.,  1.],
       [ 2.,  3.],
       [ 4.,  5.],
       [ 6.,  7.],
       [ 8.,  9.],
       [10., 11.],
       [12., 13.],
       [14., 15.],
       [16., 17.]], dtype=float32)>

In [60]:
result_flattened_shape = tf.concat([result_shape[:-2], [result_shape[-2]*result_shape[-1]]], 0)
result_flattened_shape

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([4])>

In [61]:
sparsed_shape = tf.concat([result_flattened_shape[-1:], x_tansf_shape[-2:-1], kernel.shape[-1:]], 0)
sparsed_shape = tf.cast(sparsed_shape, tf.int64)
sparsed_shape

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([4, 9, 2], dtype=int64)>

In [112]:
def iterate_sparsed_indices(sparsed_shape, input_shape, kernel_shape, result_shape, strides):
    for i in range(sparsed_shape[-3]):
        for j in range(sparsed_shape[-2]):
            for k in range(sparsed_shape[-1]):
                row_num = i // result_shape[-1]
                col_num = i % result_shape[-1]
                offset = row_num * strides * input_shape[-2] + col_num * strides
                if j >= offset and ((j - offset) // input_shape[-2]) < kernel_shape[-2] and ((j - offset) % input_shape[-2]) < kernel_shape[-2]:
                    yield [i, j, k]
                continue

# sparsed_indices = [
#     [i, j, k]
#     for i in range(result_flattened_shape[-1])
#     for j in range(x_tansf_shape[-2])
#     for k in range(kernel.shape[-1])
#     if j >= i * strides and ((j - i * strides) // x.shape[-2]) < kernel.shape[-2] and ((j - i * strides) % x.shape[-2]) < kernel.shape[-2]
# ]
sparsed_indices = list(iterate_sparsed_indices(tf.cast(sparsed_shape, tf.int32), x.shape, kernel.shape, result_shape, strides))
# sparsed_indices

In [123]:
sparsed_values = tf.reshape(kernel, [-1])
sparsed_values = tf.tile(sparsed_values, sparsed_shape[:1])

In [113]:
sparsed_kernel = sparse.SparseTensor(sparsed_indices, sparsed_values, sparsed_shape)
sparse.to_dense(sparsed_kernel)[...,1]

<tf.Tensor: shape=(4, 9), dtype=float32, numpy=
array([[2., 4., 0., 6., 8., 0., 0., 0., 0.],
       [0., 2., 4., 0., 6., 8., 0., 0., 0.],
       [0., 0., 0., 2., 4., 0., 6., 8., 0.],
       [0., 0., 0., 0., 2., 4., 0., 6., 8.]], dtype=float32)>

In [77]:
x_transf

<tf.Tensor: shape=(9, 2), dtype=float32, numpy=
array([[ 0.,  1.],
       [ 2.,  3.],
       [ 4.,  5.],
       [ 6.,  7.],
       [ 8.,  9.],
       [10., 11.],
       [12., 13.],
       [14., 15.],
       [16., 17.]], dtype=float32)>

In [114]:
print(sparsed_kernel.shape)
print(x_transf.shape)
tf.tensordot(sparse.to_dense(sparsed_kernel), x_transf, [[1, 2], [0, 1]])

(4, 9, 2)
(9, 2)


<tf.Tensor: shape=(4,), dtype=float32, numpy=array([220., 292., 436., 508.], dtype=float32)>